In [ ]:
import os
import time

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
MODEL_BASE_NAME = "MegaClassifier_b"
VERSION = 1

DATASETS = {
    "MegaClassifier_a": "onlyDetectionsForTrain",
    "MegaClassifier_b": "emptyOriginalAnimalDetection",
    "MegaClassifier_c": "emptyNonEmptyDataset",
}

In [ ]:
EPOCHS = 10
IMAGE_SIZE = (456, 456)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [ ]:
OPTIMIZERS = {"Adam": tf.keras.optimizers.Adam()}

In [ ]:
LOSS = {"BinaryCrossentropy": tf.keras.losses.BinaryCrossentropy()}

In [ ]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

In [ ]:
BATCH_SIZES = [16, 32, 64, 128]

In [ ]:
DATASET_DIR = os.path.abspath(f"data/processed/{DATASETS[MODEL_BASE_NAME]}")

SUBVERSION = 0
for BATCH_SIZE in BATCH_SIZES:
    LOGS_PATH = os.path.abspath(
        f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    MODELS_PATH = os.path.abspath(
        f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    MODEL_NAME_h5 = f"{MODEL_BASE_NAME}_v{VERSION}_{SUBVERSION}.h5"
    MODEL_NAME_keras = f"{MODEL_BASE_NAME}_v{VERSION}_{SUBVERSION}.keras"
    MODEL_NAME_weights = f"{MODEL_BASE_NAME}_v{VERSION}_{SUBVERSION}.weights.h5"

    train_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    )
    train_images = train_datagen.flow_from_directory(
        directory=f"{DATASET_DIR}/train",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )

    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    )
    validation_images = datagen.flow_from_directory(
        directory=f"{DATASET_DIR}/validation",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )

    pretrained_model = tf.keras.applications.EfficientNetB5(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    pretrained_model.trainable = False

    model = tf.keras.Sequential(
        [
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(1, activation="sigmoid"),
        ],
        name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}",
    )

    model.compile(
        optimizer=OPTIMIZERS["Adam"],
        loss=LOSS["BinaryCrossentropy"],
        metrics=METRICS,
    )

    history = model.fit(
        train_images,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=validation_images,
        callbacks=[TensorBoard(log_dir=LOGS_PATH)],
    )
    time.sleep(0.1)

    dataframe = pd.DataFrame(history.history)
    history_path = os.path.join(LOGS_PATH, f"history_v{VERSION}.{SUBVERSION}.csv")
    dataframe.to_csv(history_path, sep=";", index=False)

    os.makedirs(MODELS_PATH, exist_ok=True)
    model.save(os.path.join(MODELS_PATH, MODEL_NAME_h5))
    model.save(os.path.join(MODELS_PATH, MODEL_NAME_keras))
    model.save_weights(os.path.join(MODELS_PATH, MODEL_NAME_weights))

    SUBVERSION += 1
    time.sleep(2)
